# Importes

In [ ]:
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

# Ejecución del proceso

In [ ]:
# Carga de datos
df_enero = pd.read_parquet("./DATA/yellow_tripdata_2023-01.parquet")
df_febrero = pd.read_parquet("./DATA/yellow_tripdata_2023-02.parquet")

## Q1. Downloading the data

In [ ]:
df_enero.shape[1]

## Q2. Computing duration

In [ ]:
# Computing 'duration' column
df_enero["duration"] = (df_enero["tpep_dropoff_datetime"] - df_enero["tpep_pickup_datetime"]).dt.total_seconds()
df_enero["duration"] = (df_enero["duration"] / 60).round(2)

In [ ]:
# Standard deviation of 'duration'
df_enero["duration"].std().round(2)

## Q3. Dropping outliers

In [ ]:
# Eliminación de datos atípicos
df_enero_no_atipicos = df_enero[df_enero["duration"].between(1, 60, inclusive = "both")]

# Proporción de datos no atípicos restantes
prop_datos_no_atipicos = df_enero_no_atipicos.shape[0] / df_enero.shape[0]
print(f'{prop_datos_no_atipicos:.2%}')

## Q4. One-hot encoding

In [ ]:
# Conversión de variables categóricas a cadenas
cols_categoricas = ["PULocationID", "DOLocationID"]

df_enero_no_atipicos[cols_categoricas] = df_enero_no_atipicos[cols_categoricas].fillna(-1)\
    .astype('int').astype('str')

# Selección de variables explicativas y de respuesta
df_entrenamiento = df_enero_no_atipicos[cols_categoricas]

In [ ]:
# Conversión del dataframe en una lista de diccionarios
dicc_df_entrenamiento = df_entrenamiento.to_dict(orient = "records")

In [ ]:
# Ajustar vectorizador de diccionarios (DictVectorizer) a datos de entrenamiento
vectorizador_dicc = DictVectorizer(sparse = False)
X_entrenamiento = vectorizador_dicc.fit_transform(dicc_df_entrenamiento)

In [ ]:
# Dimensionalidad de columnas de la matriz de características
X_entrenamiento.shape[1]

## Q5. Training a model

In [ ]:
# Entrenamiento
model = LinearRegression(n_jobs = -1)
model.fit(X = X_entrenamiento, y = df_enero_no_atipicos["duration"])

# Predicción con datos de entrenamiento
Y_pred_entrenamiento = model.predict(X_entrenamiento)

In [ ]:
root_mean_squared_error(y_true = df_enero_no_atipicos["duration"], y_pred = Y_pred_entrenamiento)